In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-31 20:14:34.430 | INFO     | SiliconDriver   - Detected 4 PCI devices
2023-10-31 20:14:34.455 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 3 device_id: 0xfaca revision: 0)
2023-10-31 20:14:34.460 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 2 device_id: 0xfaca revision: 0)
2023-10-31 20:14:34.464 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 1 device_id: 0xfaca revision: 0)
2023-10-31 20:14:34.473 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-31 20:14:34.553 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:14:34.553 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:14:34.553 | INFO  

# Configuration

In [2]:
h = 31
w = 64

# Initialize tensors a and b with random values using torch

In [3]:
torch_a = torch.rand((h, w), dtype=torch.bfloat16)
torch_b = torch.rand((h, w), dtype=torch.bfloat16)

a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          


# Add tensor a and b

In [4]:
output = a + b

                     Op | INFO     | tt::tt_metal::TilizeWithValPadding                 device                                            
                     Op | INFO     | tt::tt_metal::TilizeWithValPadding                 device                                            
                     Op | INFO     | tt::tt_metal::EltwiseBinary                        device                                            
                     Op | INFO     | tt::tt_metal::UntilizeWithUnpadding                device                                            


# Inspect the result of the add

In [5]:
print("Printing ttnn tensor")
print(output.shape)
print(output[:1])

Printing ttnn tensor
[31, 64]
                     Op | INFO     | from_device                                        external                                          
                     Op | INFO     | to_torch                                           external                                          
Tensor([ [1.21094, 0.203125, 0.753906, 0.988281, 0.925781, 1.94531, 1.32812, 0.632812, 0.101562, 1.59375, 0.316406, 1.54688, 0.695312, 0.765625, 0.511719, 1.55469, 0.660156, 0.660156, 1, 0.863281, 1.6875, 0.671875, 1.23438, 1.57031, 0.671875, 1.03125, 1.59375, 0.792969, 0.636719, 0.988281, 0.0742188, 0.746094, 1.17969, 1.42969, 1.17969, 1.32812, 1.94531, 0.996094, 1.15625, 1.24219, 0.539062, 1.14062, 1.25, 1.21875, 1.71875, 0.671875, 1.03906, 1.48438, 0.757812, 0.777344, 0.464844, 1.5625, 0.667969, 0.835938, 0.695312, 0.972656, 0.828125, 0.804688, 1.70312, 1.07031, 1.24219, 1.82031, 1.3125, 0.921875]], dtype=bfloat16 )

                     Op | INFO     | _TensorBase.__getitem__    

# Close the device

In [6]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
